In [31]:
import numpy as np
import os

def read_npz(filepath):
    if os.path.exists(filepath):
        try:
            # 使用 numpy.load() 加载 .npz 归档文件
            event_archive = np.load(filepath)

            # 1. 查看这个归档文件里都存储了哪些数组
            print(f"✅ 归档文件 '{filepath}' 加载成功。")
            print(f"   文件中包含的数组名: {event_archive.files}")

            # 2. 访问我们关心的 'data' 数组
            if 'data' in event_archive:
                data_array = event_archive['data']

                # 3. 打印这个数组的关键属性
                print("\n--- 'data' 数组的详细信息 ---")
                print(f"形状 (Shape): {data_array.shape}  (格式为 '通道, 高度, 宽度')")
                print(f"数据类型 (Data Type): {data_array.dtype}")

                # 4. 显示数组的一小部分，直观感受里面的数值
                print("\n--- 数值抽样 (左上角 [:, :5, :5] 区域) ---")
                print(data_array[:, :5, :5])
                
            else:
                print("\n⚠️ 警告: 文件中没有找到名为 'data' 的数组。")

        except Exception as e:
            print(f"\n❌ 读取文件时发生错误: {e}")
    else:
        print(f"❌ 错误: 在路径 '{filepath}' 未找到文件，请检查路径是否正确。")

In [2]:
filepath1 = 'GoPro_event.npz'
filepath2 = 'HQEVFI_event.npz'

In [3]:
read_npz(filepath1)

✅ 归档文件 'GoPro_event.npz' 加载成功。
   文件中包含的数组名: ['x', 'y', 't', 'p']

⚠️ 警告: 文件中没有找到名为 'data' 的数组。


In [4]:
read_npz(filepath2)

✅ 归档文件 'HQEVFI_event.npz' 加载成功。
   文件中包含的数组名: ['sync_rgb', 'x', 'y', 'p', 't', 'start_t', 'end_t', 'sync_rgb_shape', 'sync_evs_shape']

⚠️ 警告: 文件中没有找到名为 'data' 的数组。


In [5]:
event1 = np.load(filepath1)
print(event1['x'].shape)
#print(event1['y'].shape)
# print(event1['p'].shape)
print(event1['t'])
#print(event1['p'][:100])

i=0
while event1['t'][i] < event1['t'][0]+8000:
    i+=1

print(i)

(755721,)
[  73206   73235   73441 ... 4166513 4166513 4166564]
104


In [6]:
event2 = np.load(filepath2)
print(event2['sync_rgb'].shape)
print(event2['x'].shape)
print(event2['t'])
print(event2['start_t'])
print(event2['end_t'])
print(event2['sync_rgb_shape'])
print(event2['sync_evs_shape'])



(618, 773, 3)
(6259,)
[718174. 718174. 718175. ... 725210. 725215. 725215.]
718174.0
725215.0
[618 773   3]
[  1 618 773]


In [7]:
event1 = np.load(filepath1)
print(event1['x'].shape)
#print(event1['y'].shape)
# print(event1['p'].shape)
print(event1['t'])
#print(event1['p'][:100])

time_mask = (event1['t'] >= event1['t'][0]) & (event1['t'] < event1['t'][0]+8000)
print(time_mask)

print(event1['x'][time_mask])

(755721,)
[  73206   73235   73441 ... 4166513 4166513 4166564]
[ True  True  True ... False False False]
[   8   11   13   12   11   12   12   13 1131 1183 1183 1183   10    9
   14    3   15   14   13   12    2    1 1183 1183    0   10   14 1183
   12    9 1182 1190 1145 1259 1182 1258 1131    6 1261    8 1262 1181
 1257 1263 1182   10 1147 1260  263  266  263  129  263  263   13 1195
 1256 1130 1254 1148  266  263   12 1198 1198  263    5 1180 1234  260
 1018 1140  127  364 1255  356  357  358  363  292  263  129  263  354
  355  359  360  361  362 1196  353   49  290  412 1147 1163  352  128
  291 1254  287  351 1265   26]


In [8]:
from PIL import Image
import numpy as np
from torchvision.transforms import ToTensor

totensor = ToTensor()

def imreader(impath):
    im = totensor(Image.open(impath))
    return im

impath = 'frame_000323.png'
im = imreader(impath)
print(im.shape)

torch.Size([3, 720, 1280])


In [9]:
event1 = np.load('GoPro_event.npz')
x, y, t, p = np.float32(event1['x']), np.float32(event1['y']), np.float32(event1['t']), np.float32(event1['p'])

h, w = im.shape[1:]
print(h, w)


720 1280


In [13]:
import numpy as np
def sample_events_to_grid(voxel_channels, h, w, x, y, t, p):
    voxel = np.zeros((voxel_channels, h, w), dtype=np.float32)
    # if len(t) == 0:
    #     return voxel
    t_start = t[0]
    t_end = t[-1]
    t_step = (t_end - t_start + 1) / voxel_channels
    for d in range(len(x)):
        d_x, d_y, d_t, d_p = x[d], y[d], t[d], p[d]
        d_x_low, d_y_low = int(d_x), int(d_y)
        d_t = d_t - t_start

        x_weight = d_x - d_x_low
        y_weight = d_y - d_y_low
        ind = int(d_t // t_step)
        # pv = d_p * 2 - 1
        pv = d_p
        if d_y_low < h and d_x_low < w:
            voxel[ind, d_y_low, d_x_low] += (1 - x_weight) * (1 - y_weight) * pv
        if d_y_low + 1 < h and d_x_low < w:
            voxel[ind, d_y_low + 1, d_x_low] += (1 - x_weight) * y_weight * pv
        if d_x_low + 1 < w and d_y_low < h:
            voxel[ind, d_y_low, d_x_low + 1] += (1 - y_weight) * x_weight * pv
        if d_y_low + 1 < h and d_x_low + 1 < w:
            voxel[ind, d_y_low + 1, d_x_low + 1] += x_weight * y_weight * pv
    return voxel

In [14]:
voxel = sample_events_to_grid(128, h, w, x, y, t, p)
print("输出类型:", type(voxel))
print("输出形状:", voxel.shape)
print("输出数据类型:", voxel.dtype)
print("输出示例 (前几个值):")
piece = voxel[0,:,:]
print(piece.shape)
print(piece.flatten().shape)
print(piece.flatten())
print("非零元素：")
# 这行代码的意思是：打印piece数组中所有非零元素的形状（即有多少个非零元素）。
print(piece.flatten().shape)
print("非零元素：",piece.flatten()[piece.flatten() != 0].shape)
print("非0和非1的元素：",piece.flatten()[(piece.flatten() != 0) & (piece.flatten() != 1)].shape)
print("非0和非-1的元素：",piece.flatten()[(piece.flatten() != 0) & (piece.flatten() != -1)].shape)

输出类型: <class 'numpy.ndarray'>
输出形状: (128, 720, 1280)
输出数据类型: float32
输出示例 (前几个值):
(720, 1280)
(921600,)
[0. 0. 0. ... 0. 0. 0.]
非零元素：
(921600,)
非零元素： (2044,)
非0和非1的元素： (973,)
非0和非-1的元素： (1071,)


In [15]:
def events_reader(self, events_path, h, w, interp_ratio):
        evs_data = [np.load(ep) for ep in events_path]
        evs_voxels = []
        for ed in evs_data:
            evs_voxels.append(
                sample_events_to_grid(self.echannel//interp_ratio, h, w, np.float32(ed['x']), np.float32(ed['y']), np.float32(ed['t']),
                                      np.float32(ed['p'])))
        return torch.tensor(np.concatenate(evs_voxels, 0))

In [17]:
import torch
def ereader(self, events_path, h, w, interp_ratio):
        total_event_channels = self.events_channel #事件通道数128
        num_event_files = len(events_path) #事件npz文件数量
        if num_event_files == 0:
            return torch.zeros(total_event_channels, h, w)
        channels_per_event = total_event_channels // num_event_files
        
        evs_data = [np.load(ep) for ep in events_path]
        evs_voxels = []
        for ed in evs_data:
            x, y, t, p = np.float32(ed['x']), np.float32(ed['y']), np.float32(ed['t']), np.float32(ed['p'])
            if len(t) > 0:
                start_t = t[0]
                # Set a fixed duration window of 8000 microseconds (8ms)
                time_window = 8000 
                end_t = start_t + time_window
                
                # Filter events to be within this fixed window
                time_mask = (t >= start_t) & (t < end_t)
                x, y, t, p = x[time_mask], y[time_mask], t[time_mask], p[time_mask]

            voxel = sample_events_to_grid(channels_per_event, h, w, x, y, t, p)
            # 下面这一行的作用是：将由 sample_events_to_grid 函数生成的 numpy 格式体素(voxel)数据转换为 PyTorch 的张量（Tensor），
            # 并添加到 evs_voxels 列表中。这样后续可以将所有事件体素拼接成一个多通道的事件张量，便于神经网络处理。
            evs_voxels.append(torch.from_numpy(voxel))
            
        output_tensor = torch.cat(evs_voxels, 0)

        # Final check to ensure the channel dimension is exactly as expected.
        # This handles cases where total_event_channels is not perfectly divisible by num_event_files.
        current_channels = output_tensor.shape[0]
        if current_channels != total_event_channels:
            if current_channels > total_event_channels:
                # Truncate if we have too many channels
                output_tensor = output_tensor[:total_event_channels, :, :]
            else:
                # Pad with zeros if we have too few
                padding_needed = total_event_channels - current_channels
                padding = torch.zeros(padding_needed, h, w, device=output_tensor.device)
                output_tensor = torch.cat([output_tensor, padding], dim=0)

        return output_tensor

In [26]:
import numpy as np

raw_path = '816_612_8_0000000000.raw'

im=np.fromfile(raw_path, np.uint16).reshape(816, 612)
im.shape

ValueError: cannot reshape array of size 249696 into shape (816,612)

In [17]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

def parse_aps_info(aps_info_path):
    """从APS日志文件中解析出所有图像帧的时间戳。"""
    df = pd.read_csv(aps_info_path, comment='#')
    # 返回一个包含所有时间戳的列表
    return df['timestamp'].tolist()

def parse_evs_info(evs_info_path):
    """从EVS日志文件中解析出每个evs_raw文件的元数据。"""
    df = pd.read_csv(evs_info_path, comment='#')
    # 返回一个字典列表，每个字典包含文件名和起止时间
    return df['timestamp'].tolist()

In [19]:
aps_info_path = 'quadbayer_10bit_3264_2448_20250703093040412_info.txt'
aps_timestamp=parse_aps_info(aps_info_path)
print(len(aps_timestamp))

evs_info_path = 'normal_v2_816_612_20250703093040412_info.txt'
evs_timestamp=parse_evs_info(evs_info_path)
print(len(evs_timestamp))


200
7800


In [ ]:
import numpy as np

def parse_evs_raw(file_path):
    """
    根据APX EVB Gen2手册，解析单个EVS .raw/.dat文件。
    返回一个 (N, 4) 的Numpy数组，列分别为 (x, y, t, p)。
    """
    # 初始化一个空列表，用于收集从各个数据包中解析出的事件数组
    all_events = []
    
    # 'with open(...) as f:' 是一种安全的文件操作方式，能确保文件最终被正确关闭
    # 'rb' 表示以“二进制读取”(read binary)模式打开文件，因为事件文件不是文本
    with open(file_path, 'rb') as f:
        
        # --- 文件头处理 ---
        # 代码作用: 将文件的读取指针从开头向后移动1024个字节。
        # 设计依据: 手册第6页，“3.1 数据采集”章节中提到，
        #           “.dat 文件都包含文件头”，且文件头大小为1024字节。
        #           我们不关心文件头内容，所以直接跳过。
        f.seek(1024)
        
        # --- 循环读取数据包 ---
        # 代码作用: 启动一个无限循环，持续从文件中读取数据，直到文件末尾。
        # 设计依据: 手册第6页指出，文件头之后是“N 个 EVS 数据包”的连续存储。
        #           这个循环结构就是为了逐个处理这些数据包。
        while True:
            # --- 解析包头 ---
            # 代码作用: 从当前指针位置读取8个字节。
            # 设计依据: 手册第7页，“EVS 数据包头格式”表格显示，包头大小为64位，即8字节。
            packet_header_bytes = f.read(8)
            
            # 代码作用: 如果读取不到任何字节，说明已经到达文件末尾，跳出循环。
            if not packet_header_bytes:
                break
                
            # 代码作用: 使用Numpy的frombuffer功能，将8字节的二进制数据块直接转换成一个64位无符号整数。
            header_data = np.frombuffer(packet_header_bytes, dtype=np.uint64)[0]
            
            # 代码作用: 使用位运算“与”(&)和一个掩码(mask)，提取出低56位作为时间戳的高位。
            #           0x00FFFFFFFFFFFFFF 是一个56位的全1掩码。
            # 设计依据: 依据手册第7页的包头格式表，高56位是 timestamp。
            timestamp_high = header_data & 0x00FFFFFFFFFFFFFF
            
            # 代码作用: 使用位运算“右移”(>>)，将64位整数向右移动56位，使得最高的8位移动到最低的位置，
            #           然后再用掩码 0xFF (二进制的11111111) 提取出这8位数，即为事件数量。
            # 设计依据: 依据手册第7页的包头格式表，紧跟时间戳之后的是8位的 event_num。
            event_num = (header_data >> 56) & 0xFF
            
            if event_num == 0:
                continue # 如果这个包没有事件，就处理下一个包
            
            # --- 解析事件数据 ---
            # 代码作用: 根据包头中得到的事件数 event_num，读取 N * 4 字节的数据块。
            # 设计依据: 手册第7页，“32bit 事件数据格式”说明每个事件占用32位，即4字节。
            event_data_bytes = f.read(event_num * 4)
            
            # 代码作用: 将整个事件数据块一次性转换成一个Numpy数组，每个元素是32位无符号整数。
            #           这比用循环逐个转换4字节要高效得多。
            events_int32 = np.frombuffer(event_data_bytes, dtype=np.uint32)
            
            # --- 解码单个事件 (位运算) ---
            # 这是最核心的解码步骤，完全复刻了手册第7页的“32bit 事件数据格式”表格。
            # p (极性): 右移31位，取最低1位。
            p = (events_int32 >> 31) & 0x01
            # y (Y坐标): 右移20位，取11位 (掩码0x7FF = 二进制11个1)。
            y = (events_int32 >> 20) & 0x7FF
            # x (X坐标): 右移8位，取12位 (掩码0xFFF = 二进制12个1)。
            x = (events_int32 >> 8)  & 0xFFF
            # timestamp_low (时间戳低位): 不移位，直接取最低的8位 (掩码0xFF)。
            timestamp_low = events_int32 & 0xFF
            
            # --- 合并与存储 ---
            # 代码作用: 将从包头获得的时间戳高位与从每个事件中获得的低8位时间戳合并。
            #           最简单的合并方式是直接相加，这通常是设计者意图的。
            # 设计依据: 手册中分别定义了高位和低位时间戳，暗示了它们需要被组合以获得完整精度。
            full_timestamp = timestamp_high + timestamp_low
            
            # 代码作用: 使用np.stack将解码出的x, y, t, p这四个数组按列堆叠，
            #           形成一个 (event_num, 4) 形状的数组。
            packet_events = np.stack([
                x.astype(np.float32), 
                y.astype(np.float32), 
                full_timestamp.astype(np.float32), 
                p.astype(np.float32)
            ], axis=1)
            
            # 代码作用: 将这个数据包解析出的事件数组，添加到总列表中。
            all_events.append(packet_events)
            
    # --- 函数返回 ---
    # 代码作用: 检查总列表是否为空，如果为空则返回一个空的(0, 4)数组。
    if not all_events:
        return np.empty((0, 4), dtype=np.float32)
        
    # 代码作用: 使用np.concatenate将列表中所有数据包的事件数组，沿行(axis=0)拼接起来，
    #           形成一个包含该文件所有事件的、巨大的 (N, 4) 数组。
    return np.concatenate(all_events, axis=0)